# Where, Offset, and domain

## Conditional: where

The `where` builtin works analogously to the numpy version (https://numpy.org/doc/stable/reference/generated/numpy.where.html)

Both require the same 3 input arguments:
- mask: a field of booleans or an expression evaluating to this type
- true branch: a tuple, a field, or a scalar
- false branch: a tuple, a field, of a scalar

Take a simple numpy example, the `mask` here is a condition:

In [1]:
from helpers import *

import gt4py.next as gtx

backend = None
# backend = gtfn_cpu
# backend = gtfn_gpu

In [2]:
a_np = np.arange(10.0)
b_np = np.where(a_np < 6.0, a_np, a_np * 10.0)
print("a_np array: {}".format(a_np))
print("b_np array: {}".format(b_np))

a_np array: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
b_np array: [ 0.  1.  2.  3.  4.  5. 60. 70. 80. 90.]


### **Task**: replicate this example in gt4py

In [3]:
@gtx.field_operator
def fieldop_where(a: gtx.Field[Dims[K], float]) -> gtx.Field[Dims[K], float]:
    return where(a < 6.0, a, a * 10.0)


@gtx.program(backend=backend)
def program_where(a: gtx.Field[Dims[K], float], b: gtx.Field[Dims[K], float]):
    fieldop_where(a, out=b)

In [4]:
def test_where():
    a = gtx.as_field([K], np.arange(10.0), allocator=backend)
    b = gtx.as_field([K], np.zeros(shape=10), allocator=backend)
    program_where(a, b, offset_provider={})

    assert np.allclose(b_np, b.asnumpy())

In [5]:
test_where()
print("Test successful")

Test successful


/var/folders/2b/_2y31vzs4sl_7rngh2yghbpw0000gn/T/ipykernel_56692/2672671145.py:4: UserWarning: Field View Program 'program_where': Using Python execution, consider selecting a perfomance backend.
  program_where(a, b, offset_provider={})


## Domain

The same operation can be performed in gt4py by including the `domain` keyowrd argument on `field_operator` call

### **Task**: implement the same operation as above using `domain` instead of `where`

In [6]:
@gtx.field_operator
def fieldop_domain(a: gtx.Field[Dims[K], float]) -> gtx.Field[Dims[K], float]:
    return a * 10.0


@gtx.program(backend=backend)
def program_domain(a: gtx.Field[Dims[K], float], b: gtx.Field[Dims[K], float]):
    fieldop_domain(a, out=b, domain={K: (6, 10)})

In [7]:
def test_domain():
    a = gtx.as_field([K], np.arange(10.0), allocator=backend)
    b = gtx.as_field([K], np.arange(10.0), allocator=backend)
    program_domain(a, b, offset_provider={})

    assert np.allclose(b_np, b.asnumpy())

In [8]:
test_domain()
print("Test successful")

Test successful


/var/folders/2b/_2y31vzs4sl_7rngh2yghbpw0000gn/T/ipykernel_56692/2292331765.py:4: UserWarning: Field View Program 'program_domain': Using Python execution, consider selecting a perfomance backend.
  program_domain(a, b, offset_provider={})


## where and domain

A combination of `where` and `domain` is useful in cases when an offset is used which exceeds the field size.

e.g. a field `a: gtx.Field[Dims[K], float]` with shape (10,) is applied an offset (`Koff`).

### **Task**: combine `domain` and `where` to account for extra indices

Edit the code below such that:
 1. operations on field `a` are performed only up until the 8th index
 2. the domain is properly set accound for the offset

#### Python reference

In [9]:
a_np_result = np.zeros(shape=10)
for i in range(len(a_np)):
    if a_np[i] < 8.0:
        a_np_result[i] = a_np[i + 1] + a_np[i]
    elif i < 9:
        a_np_result[i] = a_np[i]
print("a_np_result array: {}".format(a_np_result))

a_np_result array: [ 1.  3.  5.  7.  9. 11. 13. 15.  8.  0.]


In [10]:
@gtx.field_operator
def fieldop_domain_where(a: gtx.Field[Dims[K], float]) -> gtx.Field[Dims[K], float]:
    return where(a < 8.0, a(Koff[1]) + a, a)


@gtx.program(backend=backend)
def program_domain_where(a: gtx.Field[Dims[K], float], b: gtx.Field[Dims[K], float]):
    fieldop_domain_where(a, out=b, domain={K: (0, 9)})

In [11]:
def test_domain_where():
    a = gtx.as_field([K], np.arange(10.0), allocator=backend)
    b = gtx.as_field([K], np.zeros(shape=10), allocator=backend)
    program_domain_where(a, b, offset_provider={"Koff": K})

    assert np.allclose(a_np_result, b.asnumpy())

In [12]:
test_domain_where()
print("Test successful")

Test successful


/var/folders/2b/_2y31vzs4sl_7rngh2yghbpw0000gn/T/ipykernel_56692/57164705.py:4: UserWarning: Field View Program 'program_domain_where': Using Python execution, consider selecting a perfomance backend.
  program_domain_where(a, b, offset_provider={"Koff": K})
